# Proyecto Final
## Data Science I: Fundamentos para la Ciencia de Datos
### Fecha: 16-07-20204
### Nombre archivo: `TMDB_tv_dataset_v3.csv`
#### source: https://www.kaggle.com/datasets/asaniczka/full-tmdb-tv-shows-dataset-2023-150k-shows?resource=download


## Objetivos
- Elegir un método de feature selection para reducir la dimensionalidad del dataset.
- Elegir un algoritmo de regresión o clasificación para entrenar con los datos elegidos.
- Validar modelo a traves de la metricas basicas. 
- Generar conclusiones con base en los resultados obtenidos.

## Metadata 
El conjunto de datos proporciona información crucial para analizar las tendencias y predecir el éxito de películas y shows de televisión utilizando información de The Movie Database (TMDB). La base de datos esta compuesta por 29 variables y mas de 150000 registros, los cuales incluyen atributos interesantes de analisis como el genero, popularidad y fecha de emision, permitiendo la segmentación y análisis detallado. Esta cantidad de datos permite una comprensión profunda y detallada de las preferencias y comportamientos de la audiencia.

Los análisis derivados de este dataset presentan una oportunidad valiosa para la industria del entretenimiento, permitiendo la identificación de factores que impulsan la popularidad, la evaluación de la efectividad de estrategias de lanzamiento y marketing, y la formulación de recomendaciones personalizadas para los usuarios, por mencionar algunos.

Este conjunto de datos es una herramienta invaluable para investigadores y profesionales de la industria del entretenimiento, contribuyendo a la mejora de la toma de decisiones estratégicas, la personalización de la experiencia del usuario y el incremento de la satisfacción y retención de la audiencia. 

##  Variables
id: id unico del show
 <br> name: nombre del show
 <br> number_of_seasons: numero de temporadas
 <br> number_of_episodes: numero de episodios 
 <br> original_language: idioma original
 <br> vote_count: cantidad de votos, 0 indica sin informacion
 <br> vote_average: nota promedio de los votos, 0 indica sin informacion
 <br> overview: resumen
 <br> adult: boolean que indica si el show es categoria adulto
 <br> backdrop_path: url para la imagen del show
 <br> first_air_date: primera fecha de emision
 <br> last_air_date: ultima fecha de emision 
 <br> homepage: url oficial del show
 <br> in_production: boolean que indica si esta en producion
 <br> original_name: nombre original 
 <br> popularity: popularidad, 0 indica sin informacion
 <br> poster_path: url para la imagen del poster
 <br> type: tipo de show
 <br> status: status 
 <br> tagline: linea asociada al show
 <br> genres: listado de genero
 <br> created_by: creado por
 <br> languages:listado idiomas disponibles
 <br> networks: listado de cadenas que han emitido el programa 
 <br> origin_country: listado de pais/es de origen
 <br> spoken_languages: listado de idiomas hablados 
 <br> production_companies: listado de compañias productoras
 <br> production_countries:listado de los paises de produccion
 <br> episode_run_time: duracion de los episodios, 0 indica no informacion

# Carga de datos

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter, AutoDateLocator,AutoDateFormatter,YearLocator
import seaborn as sns
import statsmodels.formula.api as sm

In [3]:
tmdb=pd.read_csv("../data/TMDB_tv_dataset_v3.csv")

# Análisis exploratorio de datos (EDA)
A continuacion se entregara un resumen de los puntos mas importantes obtenido en el EDA, si se desea ver en mayor detalle puede consultar el notebook ProyectoDS_ParteI_jofre.ipynb

### Renomabramiento de vaiables

In [4]:
nombres_columnas= {'id':"id",
'name':"Nombre_Serie",
'number_of_seasons':"num_Temporadas",
'number_of_episodes':"num_Episodios",
'original_language':"Idioma_original",
'vote_count':"Votos_Totales",
'vote_average':"Voto_promedio",
'overview':"Resumen",
'adult':"Categoria_Adulto",
'backdrop_path':"url_Imagen",
'first_air_date':"Fecha_Inicio",
'last_air_date':"Fecha_Fin",
'homepage':"Pagina",
'in_production':"En_Produccion",
'original_name':"Nombre_Original",
'popularity':"Popularidad",
'poster_path':"url_Poster",
'type':"Tipo_Show",
'status':"Status",
'tagline':"Frase",
'genres':"Generos",
'created_by':"Creador",
'languages':"Idiomas",
'networks':"Canales",
'origin_country':"Pais_Origen",
'spoken_languages':"Idiomas_hablados",
'production_companies':"Companias_Produccion",
'production_countries':"Paises_Produccion",
'episode_run_time':"Duracion_episodio"}

tmdb.rename(columns=nombres_columnas,inplace=True)

### Tratamiento de valores nulos
- Para la columna de generos, se rellena la columna con el valor "Sin información"
- Se eliminan las columnas que tienen soobre un 35% de valores nulos, que no entregan informacion relevante, como url de la imagen del poster. 
- Se eliminan las filas que no poseen valores de las fechas de inicio o final.
- Aún con esto se posee una base con sobre 90 mil filas de información.  

In [7]:
tmdb=tmdb.dropna(subset=['Nombre_Serie'])
tmdb['Generos']=tmdb['Generos'].fillna('sin informacion')
tmdb=tmdb.drop(columns=['Resumen','url_Imagen','url_Poster','Frase','Pagina','Creador'])
tmdb=tmdb.dropna(subset=['Fecha_Inicio','Fecha_Fin','Idiomas' ])
tmdb.shape

(98775, 28)

### Columna calculadas 
Para obtener un mejor detalle se extrae informacion temporal de las fechas, como el mes de ingreso y la estacion del año.

In [5]:
tmdb['Fecha_Inicio']= pd.to_datetime(tmdb['Fecha_Inicio'])
tmdb['Mes']= tmdb['Fecha_Inicio'].dt.month
tmdb['estacion']=tmdb['Mes']%4
tmdb['estacion_n']= tmdb['estacion'].map({0:'primavera',1:'verano',2:'otoño',3:'invierno'})
tmdb['scripted']=tmdb['Generos'].apply(lambda x: 1 if x=='Scripted' else 0)
tmdb['anho']= tmdb['Fecha_Inicio'].dt.year

Ademas dado que el tipo de programa esta desbalanceado a los del tipo script se crea una nueva variable que indica si el programa es del tipo scripted o no.

In [6]:
tmdb['scripted']=tmdb['Generos'].apply(lambda x: 1 if x=='Scripted' else 0)

## 